# **Calculations**

Here the calculations of the aiida-kkr plugin are presented. It is assumed that the user already has basic knowledge of ``python``, ``aiida`` (e.g. database structure, verdi commands, structure nodes) and KKR (e.g. LMAX cutoff, energy contour integration). Also ``aiida-kkr`` should be installed as well as the Voronoi, KKR and KKRimp codes should already be configured. Details about istallation, configuration and the most common issues that cause KKR codes are presented in the corresponding section.

## **Basic caluclations**

Here presented basic calculations, using `aiida-kkr` plugin, at the example of the bulk Cu. In practice, the use of the workflows is more convenient, which is presented in the corresponding section.

In the beggining, make sure, that `aiida` is installed, profile is configured correctly, and you are activated virtual environment. Detailed step-by-step explanation is provided in the following [AiiDA installation guide](https://aiida.readthedocs.io/projects/aiida-core/en/latest/install/installation.html).

Then, we need to load database settings and allow AiiDA to reach our database. This can be done by executing command below:

In [ ]:
from aiida import load_profile
load_profile()

### **Voronoi starting potential generator**

The Voronoi code creates starting potentials for a KKR calculation and sets up the atom-centered division of space into voronoi cells. Also corresponding shape functions are created, which are needed for full-potential corrections.

The voronoi plugin is called ``kkr.voro`` and it has the following input and output nodes:

#### **Input nodes:**
- ``parameters`` - KKR parameter set for Voronoi calculation (Dict)
- ``structure`` - structure data node, which describing the crystal lattice (StructureData)
- ``code`` - Voronoi code node

#### **Output nodes:**
- ``remote_folder`` - ...
- ``retrieved`` - ...
- ``output_parameters`` - ...

#### **Additional optional input nodes that trigger special behavior of a Voronoi calculations:**
- `parent_KKR` - ...
- `potential_overwrite` - ...

Now the basic usage of the voronoi plugin is demonstrated.

First, we create an aiida structure by getting aiida ``StructureData`` class:

In [ ]:
from aiida.plugins import DataFactory
StructureData = DataFactory('structure')

Then we create the aiida ``StructureData`` node for bulk Cu.
Defining lattice constant in Angstroem:

In [1]:
alat = 3.61

Defining Bravais matrix in Ang. units in the following form:

In [2]:
from numpy import array
bravais = alat*array([[0.5, 0.5, 0], [0.5, 0, 0.5], [0, 0.5, 0.5]])

Now create StructureData instance and set Bravais matrix and atom position in the unit cell:

In [ ]:
Cu = StructureData(cell=bravais)
Cu.append_atom(position=[0,0,0], symbols='Cu')

Next we create an empty set of KKR parameters(LMAX cutoff etc.) for voronoi code. For doing this we load ``kkrparms`` class in order ro create the set of input parameters for KKR calculations. To find out, which parameters are mandatory to be set use: 

``missing_params = params.get_missing_keys(use_aiida=True)``

In [ ]:
from masci_tools.io.kkr_params import kkrparams
params = kkrparams(params_type='voronoi')

And set at least the mandatory parameters:

In [ ]:
params.set_multiple_values(LMAX=2, NSPIN=1, RCLUSTZ=2.3)

Then we create an aiida Dict node and fill the dictionary of the parameters:

In [ ]:
Dict = DataFactory('dict')
ParaNode = Dict(dict=params.get_dict())

Now we get the voronoi code. Details about installation of the KKR codes are presented in the [Link](https://iffgit.fz-juelich.de/kkr/jukkr) in the ``ReadMe`` file.

First of all, load aiida ``Code`` class:

In [ ]:
from aiida.orm import Code

Then initialize string - the name of the code, where ``code_name`` - is the name of the code in aiida database, and ``computer`` - is the name of the computer, on which codes are set up. Details about configuring computer and setting up codes can be found in the [Aiida documentation](https://aiida.readthedocs.io/projects/aiida-core/en/latest/verdi/verdi_user_guide.html#verdi-code)

In [3]:
codename = 'code_name@computer'
code = Code.get_from_string(codename)

Now create new process builder for a Voronoi calculation:

In [ ]:
builder = code.get_builder()

and set resources that will be used (here serial job) in the options dict of the metadata:

In [ ]:
builder.metadata.options = {'resources': {'num_machines':1, 'tot_num_mpiprocs':1} }

Then set structure and input parameter:

In [ ]:
builder.structure = Cu
builder.parameters = ParaNode

Now we are ready to submit the calculation:

In [ ]:
from aiida.engine import submit
voro_calc = submit(builder)

You can also use ``run()`` method for starting the process without sending it to daemon.


In [ ]:
from aiida.engine import run
voro_calc = run(builder)

### **KKR calculation for bulk and interfaces**

A KKR calculation is provided by the ``kkr.kkr`` plugin, which has the following input and output nodes.

#### **Three input nodes:**
* ``parameters`` -  KKR parameter fitting the requirements for a KKR calculation (Dict)
* ``parent_folder`` - parent calulation remote folder node (RemoteFolder)
* ``code`` - KKR code node (code)

#### **Three output nodes:**
- ``remote_folder`` - ...(RemoteData)
- ``retrieved`` - ..(FolderData)
- ``output_parameters`` - (Dict)

#### **Special features exist where a fourth input node is persent and which triggers special behavior of the KKR calculation:**
* ``impurity_info`` - Node specifying the impurity cluster (Dict)
* ``kpoints`` -  Node specifying the kpoints for which the bandstructure is supposed to be calculated (KpointsData)
The different possible modes to run a kkr calculation (start from Voronoi calculation, continue from previous KKR calculation, host Greenfunction writeout feature) are demonstrated in the following.

### **KKR calculation from voronoi parent**

First of all, we reuse settings from previous voronoi calculation:

In [ ]:
voronoi_calc_folder = voro_calc.outputs.remote_folder
voro_params = voro_calc.inputs.parameters

This can be done both in the script as it is shown above and by downloading node from previous voronoi calculation, as it is written below:

In [ ]:
from aiida.orm import load_node
node = load_node(<pk>)
voronoi_calc_folder = node.outputs.remote_folder
voro_params = node.inputs.parameters

Inside the brackets ``<pk>`` is the id of the node of previous voronoi calculation. But keep in mind, that if you are loading node from the previous calculation, your calculation results and input data will contain in database only if you use ``submit()`` method for runninig the code.

Now we update the KKR parameter set to meet the requirements for a KKR calculation (slightly different than voronoi calculation). Thus, we create a new set of parameters for a KKR calculation and fill the already set values from the previous voronoin calculation:

New KKR parameters can be set as:

In [ ]:
from masci_tools.io.kkr_params import kkrparams
params = kkrparams(params_type='kkr', **voro_params.get_dict())

Setting the missing values:

In [ ]:
params.set_multiple_values(RMAX=7., GMAX=65.)

Choosing 20 simple mixing iterations first to preconverge potential (here 5% simple mixing):

In [ ]:
params.set_multiple_values(NSTEPS=20, IMIX=0, STRMIX=0.05)

And creating aiida dictionary (Dict) node from the KKR parameters:

In [ ]:
ParaNode = Dict(dict=params.get_dict())

Now we are getting the KKR code and create new calculation instance and set the input nodes accordingly:

In [ ]:
code = Code.get_from_string('code_name@computer')
builder = code.get_builder()
builder.parameters = ParaNode
builder.parent_folder = voronoi_calc_folder
builder.metadata.options = {'resources' :{'num_machines': 1, 'num_mpiprocs_per_machine':1}}

And we are run the KKR calculation using ``submit()`` method for keeping the calculation results in database:

In [ ]:
kkr_calc = submit(builder)

Or just running it in the terminal:

In [ ]:
kkr_calc = run(builder)

### **Continue KKR calculation from KKR parent calculation**

First we reuse setting from the previous preconverged KKR calculation:

In [ ]:
kkr_calc_parent_folder = kkr_calc.outputs.remote_folder
kkr_params = kkr_calc.inputs.parameters

Or if you submited your calculation to the daemon, it is necessary to laod the node of the previous kkr calculation:

In [ ]:
node = load_node(<pk>)
kkr_calc_parent_folder = node.outputs.remote_folder
kkr_params = node.inputs.parameters

Where ``<pk>`` is the id of the previous calculation.

Then we are  reusing the old KKR parameters and update scf settings (default is NSTEP=1,IMIX=0)

In [ ]:
params = kkrparams(params_type='kkr', **kkr_params.get_dict())
params.set_multiple_values(NSTEPS=50, IMIX=5)

Now we are getting the KKR code and create new calculation:

In [ ]:
code = Code.get_from_string('code_name@computer')
builder = code.get_builder()

Then we are creating the aiida Dict node and setting the input nodes for calculation:

In [ ]:
ParaNode = Dict(dict=params.get_dict())
builder.parameters = ParaNode
builder.parent_folder = kkr_calc_parent_folder
builder.metadata.options = {'resources': {'num_machines': 1, 'num_mpiprocs_per_machine':1}}

And running the calculation with updated set of parameters:

In [ ]:
kkr_calc_continued = run(builder)

### **Density of states**

Here is presented the density of states calculation using workflow ``kkr_dos_wc``, which automatically sets the right parameters in the input of a KKR calculation to perform a DOS calculation. The specifics of the DOS energy contour are set via the ``wf_parameters`` input node which contains default values if no user input is given

#### **Input nodes:**
- ``kkr`` - KKrcode using the ``kkr.kkr`` plugin (aiida.orm.Code)
- ``remote_data`` - The remote folder of the (converged) calculation whose output potential is used as input for the DOS run (RemoteData)
- ``wf_parameters`` - Some settings of the workflow behavior (e.g. number of energy points in DOS contour etc.) (ParameterData, optional)
- ``options`` - Some settings for the computer you want to use (e.g. queue_name, use_mpi, resources, …) (ParameterData, optional)
- ``label`` - Label of the workflow (str, optional)
- ``description`` - Longer description of the workflow (str, optional)

#### **Returns nodes:**
- ``dos_data`` - The DOS data on the DOS energy contour (i.e. at some finite temperature) (XyData)
- ``dos_data_interpol`` - The interpolated DOS from the line parallel to the real axis down onto the real axis (XyData)
- ``results_wf``  - The output node of the workflow containing some information on the DOS run (ParameterData)

We are starting from getting an installed KKRcode:

In [ ]:
from aiida.orm import Code
kkrcode = Code.get_from_string('code_name@computer')

Now we are loading the remote folder node from the converged KKR calculation from which which we want to start the following DOS calculation:

In [ ]:
from aiida.orm import load_node
kkr_remote_folder = load_node(160).outputs.remote_folder

Then we are setting of the workflow parameters and the calculation options:

In [ ]:
from aiida.plugins import DataFactory
ParameterData = DataFactory('dict')
workflow_settings = ParameterData(dict={'dos_params':{'emax': 1, 'tempr': 200, 'emin': -1,
                                                      'kmesh': [20, 20, 20], 'nepts': 81}})
calc_options = ParameterData(dict={'resources': {'num_machines': 1,\
                                                 'num_mpiprocs_per_machine':1}})

??? something about this step??? ....

In [ ]:
inputs = {
    'kkr': kkrcode,
    'remote_data': kkr_remote_folder,
    'wf_parameters': workflow_settings,
    'options': calc_options,
}

And running the DOS calculation:

In [ ]:
run(kkr_dos_wc, **inputs)